# Long Puts OTM Strategy Eval

What if I continually buy puts that are way out of the money? Do you win big enough to outweigh the small losses?

Evaluation steps:

1. Pull stocks from Xano
2. Get historical prices on each stock
3. Get option chains on each stock (is it possible to get historical chains?)
4. Simulate the stock prices
5. Choose the stocks most likely to go down by expiration date
6. "Buy" puts and see

## Pull stocks from Xano

In [24]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from scipy.stats import genextreme
from random import choices

pd.set_option('display.max_columns', None)

In [2]:
url = 'https://iocalc.com/api:gPr_YnZX/getstocks'

response = requests.get(url)

stock_table = pd.json_normalize(response.json()['stocks'])

In [3]:
stock_table

,id,created_at,name,symbol,price,target,marketcap,week52high,week52low,dividendyield,...,pe,beta,change_1yr,change_ytd,pricetargethigh,pricetargetlow,symboltype,analysts,currency,totaltracking
0,20,1669528036105,Meta Financial Group Inc.,CASH,44.66,72.50,1741265522,65.9,36.02,0.003454,...,6.2630944593554,1.1439973981583,0.48746215755319,-0.030673818303721,79.0,70.0,stock,4,USD,0
1,19,1669528050982,Global X Funds,QYLD,16.45,0.00,5871377700,22.43,19.11,0.121877,...,,,-0.0083631439208244,-0.079315006759802,0.0,0.0,,0,,0
2,11,1669528132313,Verizon Communications Inc,VZ,39.02,60.00,223153658344,58.44,49.09,0.058884,...,10.124939126336,0,-0.032254541085492,0.035421706459115,71.0,52.0,stock,26,USD,0
3,49,1669528349343,"Delta Air Lines, Inc.",DAL,35.10,51.56,24262923840,52.28,33.4,0.000000,...,-356.807703535,1.1989249496749,-0.05225,-0.029938587512794,67.0,40.0,stock,18,USD,0
4,54,1669529418716,Humana Inc.,HUM,540.95,485.87,48727349718,473.14,351.2,0.007368,...,18.229461173999,0.54182997393569,-0.058274921620204,-0.18272754710473,581.0,410.0,stock,23,USD,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,74,1643141926509,"Oceaneering International, Inc.",OII,13.88,17.72,1282413359,18.2,8.27,0.000000,...,-36.130426529836,1.8863947677163,0.21916508538899,0.13616268788683,20.0,16.0,stock,9,USD,0
66,70,1669528047177,Kosmos Energy Ltd,KOS,6.41,5.96,1916009623,4.89,1.8,0.000000,...,-11.364502259128,2.3177199524754,0.58208955223881,0.22543352601156,8.0,4.1,stock,7,USD,0
67,76,1669528829688,GlobalFoundries Inc,GFS,67.61,79.10,33128671394,79.49,43.59,0.000000,...,-132.34898464626,0,0.34245689655172,-0.041249807603509,100.0,46.0,stock,15,USD,0
68,65,1643141629684,ALPS Fund Services,SBIO,33.95,0.00,146072500,64.04,31.96,0.000000,...,,,-0.38581129378127,-0.18631628787879,0.0,0.0,,0,,0


## Get Historical Prices for Each Stock

In [4]:
config = pd.read_csv('../tradier_exploration/sandbox.config')
token = config.prod_token[0]

In [5]:
endpoint = 'https://api.tradier.com'
path = '/v1/markets/history'

all_stock_history = pd.DataFrame()

for symbol in stock_table['symbol']:
    response = requests.get(f'{endpoint}{path}',
        params={'symbol':f'{symbol}', 
                'interval': 'daily', 
                'start': '2000-01-01', 
                'end': f'{datetime.today().date().isoformat()}'},
        headers={'Authorization': f'Bearer {token}', 
                 'Accept': 'application/json'}
    )
    json_response = response.json()
    this_stock = pd.json_normalize(json_response['history']['day'])
    this_stock['symbol'] = symbol
    
    all_stock_history = pd.concat([all_stock_history, this_stock], axis = 0, ignore_index = True)

In [6]:
all_stock_history

,date,open,high,low,close,volume,symbol
0,2000-01-03,4.166667,4.166667,4.0,4.000000,5700,CASH
1,2000-01-04,3.958333,3.958333,3.833333,3.833333,21900,CASH
2,2000-01-05,3.770833,3.770833,3.770833,3.770833,2700,CASH
3,2000-01-06,3.916667,4.0,3.833333,3.916667,3300,CASH
4,2000-01-07,3.833333,3.833333,3.75,3.750000,27300,CASH
...,...,...,...,...,...,...,...
256723,2022-11-18,4.7,4.71,4.425,4.600000,2205709,RKLB
256724,2022-11-21,4.61,4.65,4.29,4.320000,3719993,RKLB
256725,2022-11-22,4.32,4.41,4.2,4.360000,2867496,RKLB
256726,2022-11-23,4.36,4.445,4.32,4.400000,3245839,RKLB


## Get Option Chains

### Get Option Expirations

In [7]:
path = '/v1/markets/options/expirations'

option_dates = pd.DataFrame()

for symbol in stock_table['symbol']:
    response = requests.get(f'{endpoint}{path}',
        params={'symbol': f'{symbol}', 
                'includeAllRoots': 'true'},
        headers={'Authorization': f'Bearer {token}', 
                 'Accept': 'application/json'}
    )
    
    these_dates = pd.DataFrame({'exp_dates': response.json()['expirations']['date'],
                                'symbol': symbol})
    option_dates = pd.concat([option_dates, these_dates], axis = 0, ignore_index = True)

In [8]:
option_dates

,exp_dates,symbol
0,2022-12-16,CASH
1,2023-01-20,CASH
2,2023-03-17,CASH
3,2023-06-16,CASH
4,2022-12-16,QYLD
...,...,...
645,2023-01-20,RKLB
646,2023-04-21,RKLB
647,2023-07-21,RKLB
648,2024-01-19,RKLB


### Get Put Chains

In [9]:
path = '/v1/markets/options/chains'

all_puts = pd.DataFrame()

for symbol in stock_table['symbol']:
    dates = option_dates.loc[option_dates['symbol'] == symbol]
    for date in dates['exp_dates']:
        response = requests.get(f'{endpoint}{path}',
            params={'symbol': f'{symbol}', 
                    'expiration': f'{date}', 
                    'greeks': 'true'},
            headers={'Authorization': f'Bearer {token}', 
                     'Accept': 'application/json'}
        )

        json_response = response.json()

        tmp = pd.json_normalize(json_response['options']['option'])
        tmp = tmp.loc[tmp['option_type'] == 'put']

        all_puts = pd.concat([all_puts, tmp], axis = 0, ignore_index = True)

In [12]:
all_puts

,symbol,description,exch,type,last,change,volume,open,high,low,close,bid,ask,underlying,strike,change_percentage,average_volume,last_volume,trade_date,prevclose,week_52_high,week_52_low,bidsize,bidexch,bid_date,asksize,askexch,ask_date,open_interest,contract_size,expiration_date,expiration_type,option_type,root_symbol,greeks.delta,greeks.gamma,greeks.theta,greeks.vega,greeks.rho,greeks.phi,greeks.bid_iv,greeks.mid_iv,greeks.ask_iv,greeks.smv_vol,greeks.updated_at,greeks
0,CASH221216P00017500,CASH Dec 16 2022 $17.50 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,0.00,0.80,CASH,17.5,NaN,0,0,0,NaN,0.0,0.0,0,Q,1669387040000,82,C,1669399080000,0,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,2.889865,2.889865,0.309,2022-11-25 17:58:39,NaN
1,CASH221216P00020000,CASH Dec 16 2022 $20.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,0.00,0.05,CASH,20.0,NaN,0,0,0,NaN,0.0,0.0,0,A,1669386617000,2,D,1669386616000,0,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,1.485829,1.485829,0.309,2022-11-25 17:58:39,NaN
2,CASH221216P00022500,CASH Dec 16 2022 $22.50 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,0.00,0.80,CASH,22.5,NaN,0,0,0,NaN,0.0,0.0,0,Q,1669387040000,81,A,1669399081000,0,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,2.202682,2.202682,0.309,2022-11-25 17:58:39,NaN
3,CASH221216P00025000,CASH Dec 16 2022 $25.00 Put,Z,option,0.7,0.0,0,NaN,NaN,NaN,NaN,0.00,0.15,CASH,25.0,0.0,0,1,1659965460356,0.7,0.0,0.0,0,W,1669396688000,37,B,1669396690000,30,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,1.307778,1.307778,0.309,2022-11-25 17:58:39,NaN
4,CASH221216P00030000,CASH Dec 16 2022 $30.00 Put,Z,option,0.15,0.0,0,NaN,NaN,NaN,NaN,0.00,0.80,CASH,30.0,0.0,0,5,1668532363544,0.15,0.0,0.0,0,Q,1669387040000,83,C,1669399081000,84,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,1.409931,1.409931,0.309,2022-11-25 17:58:39,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24762,RKLB250117P00003000,RKLB Jan 17 2025 $3.00 Put,Z,option,1.05,0.05,1,1.05,1.05,1.05,1.05,0.75,1.30,RKLB,3.0,5.0,0,1,1669392486484,1.0,0.0,0.0,212,B,1669399080000,2,A,1669392486000,15,100,2025-01-17,standard,put,RKLB,-0.158352,4.157843e-02,-1.010638e-03,0.014861,0.020589,-0.075676,0.695346,0.871985,1.048624,0.909,2022-11-25 17:58:59,NaN
24763,RKLB250117P00004000,RKLB Jan 17 2025 $4.00 Put,Z,option,1.6,0.0,0,NaN,NaN,NaN,NaN,1.25,2.15,RKLB,4.0,0.0,0,4,1669225276647,1.6,0.0,0.0,147,B,1669399169000,217,E,1669399142000,1042,100,2025-01-17,standard,put,RKLB,-0.213089,5.028727e-02,-1.204106e-03,0.017905,0.021771,-0.070742,0.663473,0.911214,1.158955,0.915,2022-11-25 17:58:59,NaN
24764,RKLB250117P00005000,RKLB Jan 17 2025 $5.00 Put,Z,option,2.3,0.0,0,NaN,NaN,NaN,NaN,2.00,2.90,RKLB,5.0,0.0,0,1,1669231198777,2.3,0.0,0.0,1,N,1669399185000,56,B,1669399185000,43,100,2025-01-17,standard,put,RKLB,-0.262191,5.648857e-02,-1.342242e-03,0.020007,0.022214,-0.066311,0.708433,0.935276,1.162119,0.924,2022-11-25 17:58:59,NaN
24765,RKLB250117P00007000,RKLB Jan 17 2025 $7.00 Put,Z,option,3.1,0.0,0,NaN,NaN,NaN,NaN,3.10,4.60,RKLB,7.0,0.0,0,3,1668609239331,3.1,0.0,0.0,135,B,1669399169000,120,B,1669399169000,4,100,2025-01-17,standard,put,RKLB,-0.350790,6.481446e-02,-1.495735e-03,0.023045,0.022092,-0.058321,0.536960,0.873376,1.209792,0.913,2022-11-25 17:58:59,NaN


### Simulate Stock Prices

In [43]:
num_samples = 10000
sample_size = 20000 
upper_scale = 0.60 
upper_shape = -0.09 
lower_scale = 0.65 
lower_shape = -0.1
today = datetime.utcnow().date()

final_options = pd.DataFrame()

for symbol in all_puts['underlying']:
    opt_chain = all_puts.loc[all_puts['underlying'] == symbol]
    opt_chain['strike_minus_last'] = opt_chain['strike'] - opt_chain['ask']
    hist_price = all_stock_history.loc[all_stock_history['symbol'] == symbol]
    hist_price['perc_change'] = ((hist_price['close'] - hist_price['open']) / hist_price['open'])
    
    for option_date in opt_chain['expiration_date']:
        this_opt_chain = opt_chain.loc[opt_chain['expiration_date'] == option_date]
        
        date = {"Date": pd.date_range(today, option_date)}

        dates = pd.DataFrame(data = date)
        dates['wday'] = dates['Date'].dt.dayofweek
        dates = dates.loc[dates['wday'] != 5]
        dates = dates.loc[dates['wday'] != 6]

        num_trading_days = dates.shape[0]

        start = today - timedelta(days = num_trading_days)
        start = start.isoformat()

        perc_pos = (
            sum(hist_price.loc[hist_price['date'] > start, 'perc_change'] > 0) / 
            len(hist_price.loc[hist_price['date'] > start, 'perc_change'])
        )

        perc_neg = 1 - perc_pos

        avg = np.mean(hist_price['perc_change'])
        std = np.std(hist_price['perc_change'])

        samp_upper = genextreme.rvs(upper_shape, 
                                    loc = avg, 
                                    scale = upper_scale * std,
                                    size = round(sample_size * perc_pos))

        samp_lower = -1 * genextreme.rvs(lower_shape, 
                                         loc = avg, 
                                         scale = lower_scale * std,
                                         size = round(sample_size * perc_neg))

        samp = np.append(samp_upper, samp_lower)
        
        del samp_upper
        del samp_lower

        sample_col = pd.Series(range(1, num_trading_days + 1)).repeat(num_samples)
        dates_col = dates['Date'].repeat(num_samples)
        sample_values = np.array(choices(samp + 1, 
                                         k = num_samples * num_trading_days
                                         )).reshape(num_trading_days, 
                                                    num_samples)

        start_price = all_stock_history['close'].iloc[-1]

        sample_values[0] = start_price

        price_paths = sample_values.cumprod(axis = 0)

        final_prices = price_paths[num_trading_days - 1]

        del price_paths

        for strp in this_opt_chain.strike:
            this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
                sum(final_prices < this_opt_chain['strike_minus_last'].loc[this_opt_chain['strike'] == strp].item()) / num_samples)

            this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)

        this_opt_chain = this_opt_chain.rename(({'strike_minus_last': 'Effective Price',
                                                 'last': 'Last Price',
                                                 'expiration_date': 'Expiration Date',
                                                 'likelihood_below': 'Llhd Blw EP',
                                                 'likelihood_strike': 'Llhd Blw Stk'}),
                                                axis = 'columns')
        
        final_options = pd.concat([final_options, opt_chain], axis = 0, ignore_index = True)

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  opt_chain['strike_minus_last'] = opt_chain['strike'] - opt_chain['ask']
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hist_price['perc_change'] = ((hist_price['close'] - hist_price['open']) / hist_price['open'])
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: Setti

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_8

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_8

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_8

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_8

...

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_8

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_8

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_8

/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_below'] = (
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_84911/3477351884.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_opt_chain.loc[this_opt_chain['strike'] == strp, 'likelihood_strike'] = (sum(final_prices < strp) / num_samples)
/var/folders/vf/7cq04w71629bcwjv6x0n0nm80000gn/T/ipykernel_8

KeyboardInterrupt: 

In [44]:
final_options

,symbol,description,exch,type,last,change,volume,open,high,low,close,bid,ask,underlying,strike,change_percentage,average_volume,last_volume,trade_date,prevclose,week_52_high,week_52_low,bidsize,bidexch,bid_date,asksize,askexch,ask_date,open_interest,contract_size,expiration_date,expiration_type,option_type,root_symbol,greeks.delta,greeks.gamma,greeks.theta,greeks.vega,greeks.rho,greeks.phi,greeks.bid_iv,greeks.mid_iv,greeks.ask_iv,greeks.smv_vol,greeks.updated_at,greeks,strike_minus_last
0,CASH221216P00017500,CASH Dec 16 2022 $17.50 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,0.80,CASH,17.5,NaN,0,0,0,NaN,0.0,0.0,0,Q,1669387040000,82,C,1669399080000,0,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,2.889865,2.889865,0.309,2022-11-25 17:58:39,NaN,16.70
1,CASH221216P00020000,CASH Dec 16 2022 $20.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,0.05,CASH,20.0,NaN,0,0,0,NaN,0.0,0.0,0,A,1669386617000,2,D,1669386616000,0,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,1.485829,1.485829,0.309,2022-11-25 17:58:39,NaN,19.95
2,CASH221216P00022500,CASH Dec 16 2022 $22.50 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,0.80,CASH,22.5,NaN,0,0,0,NaN,0.0,0.0,0,Q,1669387040000,81,A,1669399081000,0,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,2.202682,2.202682,0.309,2022-11-25 17:58:39,NaN,21.70
3,CASH221216P00025000,CASH Dec 16 2022 $25.00 Put,Z,option,0.7,0.0,0,NaN,NaN,NaN,NaN,0.0,0.15,CASH,25.0,0.0,0,1,1659965460356,0.7,0.0,0.0,0,W,1669396688000,37,B,1669396690000,30,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,1.307778,1.307778,0.309,2022-11-25 17:58:39,NaN,24.85
4,CASH221216P00030000,CASH Dec 16 2022 $30.00 Put,Z,option,0.15,0.0,0,NaN,NaN,NaN,NaN,0.0,0.80,CASH,30.0,0.0,0,5,1668532363544,0.15,0.0,0.0,0,Q,1669387040000,83,C,1669399081000,84,100,2022-12-16,standard,put,CASH,0.000000,1.770891e-15,-1.690108e-14,0.000020,0.000000,0.000000,0.000000,1.409931,1.409931,0.309,2022-11-25 17:58:39,NaN,29.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45985,CASH230616P00045000,CASH Jun 16 2023 $45.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,3.2,5.60,CASH,45.0,NaN,0,0,0,NaN,0.0,0.0,75,E,1669399080000,46,B,1669399123000,0,100,2023-06-16,standard,put,CASH,-0.400176,2.938224e-02,-1.616236e-02,0.128676,0.116951,-0.149677,0.283715,0.376994,0.470273,0.393,2022-11-25 17:58:39,NaN,39.40
45986,CASH230616P00050000,CASH Jun 16 2023 $50.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,6.8,8.50,CASH,50.0,NaN,0,0,0,NaN,0.0,0.0,31,A,1669399183000,40,B,1669399187000,0,100,2023-06-16,standard,put,CASH,-0.554103,3.124103e-02,-1.484711e-02,0.133824,0.091090,-0.111267,0.323158,0.389887,0.456615,0.380,2022-11-25 17:58:39,NaN,41.50
45987,CASH230616P00055000,CASH Jun 16 2023 $55.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,10.4,12.20,CASH,55.0,NaN,0,0,0,NaN,0.0,0.0,43,B,1669399082000,40,B,1669399082000,0,100,2023-06-16,standard,put,CASH,-0.692610,2.838089e-02,-1.230585e-02,0.116767,0.064839,-0.076705,0.281838,0.370969,0.460101,0.374,2022-11-25 17:58:39,NaN,42.80
45988,CASH230616P00060000,CASH Jun 16 2023 $60.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,NaN,14.1,16.80,CASH,60.0,NaN,0,0,0,NaN,0.0,0.0,119,E,1669399141000,41,B,1669399088000,0,100,2023-06-16,standard,put,CASH,-0.797121,2.293053e-02,-9.569565e-03,0.095837,0.043708,-0.050625,0.000000,0.510875,0.510875,0.361,2022-11-25 17:58:39,NaN,43.20


In [41]:
strp

17.5